In [ ]:
import requests
import os
from decimal import Decimal
from datetime import datetime

import pandas as pd

# Calculate Stats

In [ ]:
api_token = os.environ.get("TILTIFY_TOKEN")
campaign_id = 155357

In [ ]:
rewards_dict = {}

url = f"/api/v3/campaigns/{campaign_id}/rewards?count=100"

response = requests.get(
    "https://tiltify.com" + url,
    headers={"Authorization": "Bearer {}".format(api_token)},
)
response = response.json()

for row in response["data"]:
    rewards_dict[row["id"]] = row

In [ ]:
rewards_dict[132555]

In [ ]:
donations_dict = {}

url = f"/api/v3/campaigns/{campaign_id}/donations?count=100"

while url:
    print(url)
    response = requests.get(
        "https://tiltify.com" + url,
        headers={"Authorization": "Bearer {}".format(api_token)},
    )

    response = response.json()

    for row in response["data"]:
        donations_dict[row["id"]] = row

    url = response["links"]["prev"]

In [ ]:
finished_refreshing = datetime.now()

In [ ]:
donations = pd.DataFrame.from_records(list(donations_dict.values()))
donations["rewardId"] = donations["rewardId"].astype("Int64")
donations["rewardName"] = (
    donations["rewardId"]
    .map(lambda x: None if pd.isna(x) else rewards_dict[x]["name"])
    .astype("string")
)
donations["rewardPrice"] = (
    donations["rewardId"]
    .map(lambda x: None if pd.isna(x) else rewards_dict[x]["amount"])
    .astype(float)
)
donations

In [ ]:
print("Total donations: ", len(donations))

In [ ]:
donations["is_anonymous"] = donations["name"] == "Anonymous"
anonymous = donations.groupby("is_anonymous").aggregate(
    count=("is_anonymous", "count"), sum=("amount", "sum")
)
anonymous.reset_index(inplace=True)
anonymous.rename(
    columns={"sum": "Raised", "count": "Count", "is_anonymous": "Who"}, inplace=True
)
anonymous["Who"] = anonymous["Who"].map(lambda x: "Anonymous" if x else "Other")
anonymous.sort_values("Who", inplace=True)
anonymous

In [ ]:
rewards_stats = donations.groupby("rewardId", dropna=False).aggregate(
    count=("id", "count"), raised=("amount", "sum")
)
rewards_stats["name"] = rewards_stats.index.map(
    lambda x: None if pd.isna(x) else rewards_dict[x]["name"]
).astype("string")
rewards_stats["basePrice"] = rewards_stats.index.map(
    lambda x: None if pd.isna(x) else rewards_dict[x]["amount"]
).astype(float)
rewards_stats.reset_index(inplace=True, drop=False)
rewards_stats["raisedOverPrice"] = rewards_stats["raised"] - (
    rewards_stats["basePrice"] * rewards_stats["count"]
)
rewards_stats["raisedOfTotal"] = round(
    rewards_stats["raised"] / rewards_stats["raised"].sum(), 4
)
rewards_stats.sort_values(by="raisedOfTotal", inplace=True, ascending=False)
rewards_stats.reset_index(drop=True, inplace=True)
rewards_stats.rename(
    columns={
        "rewardId": "id",
    },
    inplace=True,
)
rewards_stats = rewards_stats[
    ["id", "name", "basePrice", "count", "raised", "raisedOverPrice", "raisedOfTotal"]
]
rewards_stats["id"] = rewards_stats["id"].astype("string").fillna("")
rewards_stats["name"] = rewards_stats["name"].fillna("<No Reward>")
rewards_stats.rename(
    columns={
        "id": "id",
        "name": "Name",
        "basePrice": "Base Price",
        "count": "Count",
        "raised": "Raised",
        "raisedOverPrice": "Raised Over Base",
        "raisedOfTotal": "Percentage of Total",
    },
    inplace=True,
)
rewards_stats

In [ ]:
donations["decimals"] = (
    donations["amount"]
    .map(Decimal)
    .map(lambda x: x.quantize(Decimal(".01")))
    .map(lambda x: x % 1)
    .astype("string")
    .str.split(".", expand=True)[1]
)
decimals = (
    donations.groupby("decimals")
    .agg(count=("id", "count"))
    .reset_index()
    .sort_values("count", ascending=False)
)
decimals.rename(
    columns={"decimals": "Value after decimal point", "count": "Count"}, inplace=True
)
decimals

# Populate the Spreadsheet

In [ ]:
import pygsheets
from pygsheets.datarange import DataRange
from datetime import datetime
from string import ascii_uppercase

In [ ]:
def get_sheet(document, sheet_name):
    return document.worksheets("title", sheet_name)[0]

In [ ]:
def set_number_formatting(sheet, column, format_type, pattern):
    model_cell = pygsheets.Cell("A1")

    model_cell.set_number_format(format_type=format_type, pattern=pattern)

    pygsheets.DataRange(column, column, worksheet=sheet).apply_format(model_cell)

In [ ]:
def populate_dataframe_to_sheet(document_url_, sheet_name, df, formatting, widths):
    document = gc.open_by_url(document_url_)
    sheet = get_sheet(document, sheet_name)
    sheet.clear(fields="*")

    print("Processing", sheet_name)

    for i in range(1, 11):
        sheet.adjust_column_width(start=i, end=i, pixel_size=100)

    sheet.update_values("A1", [list(df.columns)])
    values = df.values.tolist()
    values = [[None if pd.isna(x) else x for x in row] for row in values]
    sheet.update_values("A2", values)

    position = len(df.columns) + 1
    sheet.update_value(f"{ascii_uppercase[position]}1", "Last updated")
    sheet.adjust_column_width(start=position + 1, end=position + 1, pixel_size=100)
    sheet.update_value(
        f"{ascii_uppercase[position + 1]}1", finished_refreshing.isoformat()
    )
    sheet.adjust_column_width(start=position + 2, end=position + 2, pixel_size=150)

    print("Added data")

    # set number format
    for column, (fmt, pattern) in formatting.items():
        set_number_formatting(
            sheet,
            ascii_uppercase[list(df.columns).index(column)],
            fmt,
            pattern,
        )

    print("Set number format")

    # pretty header
    header_range = DataRange(
        start="A1",
        end=f"{ascii_uppercase[len(df.columns) - 1]}1",
        worksheet=sheet,
    )
    header_range.update_borders(bottom=True, style="SOLID")

    print("Set header borders")

    for column_name in df.columns:
        column = ascii_uppercase[list(df.columns).index(column_name)]
        header_cell = sheet.cell(f"{column}1").set_text_format("bold", True)
        if column_name in formatting:
            header_cell.set_horizontal_alignment(
                pygsheets.custom_types.HorizontalAlignment.RIGHT
            )
        header_cell.update()

    print("Set header styles")

    # sheet width
    for column, width in widths.items():
        sheet.adjust_column_width(
            start=list(df.columns).index(column) + 1,
            end=list(df.columns).index(column) + 1,
            pixel_size=width,
        )

    print("Set columns widths")

In [ ]:
gc = pygsheets.authorize()

In [ ]:
import os

document_url = os.environ.get(
    "DOCUMENT_URL",
    "https://docs.google.com/spreadsheets/d/1q2i2dycsXRrYHofHXNCL9q1XtTxmTQYo2bUyxvStHmI/edit#gid=0",
)

In [ ]:
populate_dataframe_to_sheet(
    document_url,
    "Anonymous Statistics",
    anonymous,
    {
        "Count": (pygsheets.FormatType.NUMBER, "0"),
        "Raised": (pygsheets.FormatType.NUMBER, "0.00"),
    },
    {"Who": 150, "Count": 70, "Raised": 70},
)

In [ ]:
populate_dataframe_to_sheet(
    document_url,
    "Rewards Statistics",
    rewards_stats,
    {
        "id": (pygsheets.FormatType.NUMBER, "0"),
        "Count": (pygsheets.FormatType.NUMBER, "0"),
        "Base Price": (pygsheets.FormatType.NUMBER, "0.00"),
        "Raised": (pygsheets.FormatType.NUMBER, "0.00"),
        "Raised Over Base": (pygsheets.FormatType.NUMBER, "0.00"),
        "Percentage of Total": (pygsheets.FormatType.PERCENT, "0.00%"),
    },
    {
        "id": 70,
        "Name": 350,
        "Raised Over Base": 150,
        "Percentage of Total": 150,
    },
)

In [ ]:
populate_dataframe_to_sheet(
    document_url,
    "Decimals Statistics",
    decimals,
    {
        "Count": (pygsheets.FormatType.NUMBER, "0"),
        "Value after decimal point": (pygsheets.FormatType.NUMBER, "00"),
    },
    {"Count": 70, "Value after decimal point": 200},
)

## Decimal War

In [ ]:
comments_rows = []
for donation in donations_dict.values():
    comments_rows.append(
        {
            "Amount In Pennies": int(round(donation["amount"] * 100)),
            "Time (UTC)": datetime.utcfromtimestamp(donation["completedAt"] / 1000),
        }
    )
comments = pd.DataFrame(comments_rows)
comments = comments.sort_values("Time (UTC)")
comments

In [ ]:
comments["Tiltify Total"] = comments["Amount In Pennies"].cumsum()
comments["Tiltify Total Decimal Points"] = (
    comments["Tiltify Total"] % 100
).map(lambda x: "{:02d}".format(x))
times_and_decimals = comments[["Time (UTC)", "Tiltify Total Decimal Points"]].rename(
    columns={"Time (UTC)": "time", "Tiltify Total Decimal Points": "decimals"}
)
times_and_decimals

In [ ]:
from collections import Counter

In [ ]:
def get_decimal_counter(df):

    start = df.iloc[0].time
    current_value = None

    counter = Counter()

    for row in df.itertuples():
        if row.decimals != current_value:
            if current_value != None:
                counter[current_value] += (row.time - start).total_seconds() / 60

            current_value = row.decimals
            start = row.time

    counter_df = pd.DataFrame(
        list(counter.items()), columns=["Total After Decimal", "Minutes"]
    ).sort_values("Minutes", ascending=False)
    counter_df["Minutes"] = counter_df["Minutes"].round(2)
    counter_df["Percentage Of Total"] = (
        counter_df["Minutes"] / counter_df["Minutes"].sum()
    )

    return counter_df[:10].reset_index(drop=True)

In [ ]:
decimal_all = get_decimal_counter(times_and_decimals)
decimal_all

In [ ]:
decimal_stream = get_decimal_counter(
    times_and_decimals[
        times_and_decimals["time"] >= pd.Timestamp(datetime(2022, 2, 25, 17, 0, 0))
    ]
)
decimal_stream

In [ ]:
decimal_top_ten = pd.concat(
    [
        decimal_all,
        decimal_stream.rename(
            columns={
                "Total After Decimal": "Total After Decimal (stream)",
                "Minutes": "Minutes (stream)",
                "Percentage Of Total": "Percentage Of Total (stream)",
            }
        ),
    ],
    axis=1,
)
decimal_top_ten

In [ ]:
populate_dataframe_to_sheet(
    document_url,
    "Decimal War Statistics",
    decimal_top_ten,
    {
        "Total After Decimal": (pygsheets.FormatType.NUMBER, "00"),
        "Minutes": (pygsheets.FormatType.NUMBER, "0.0"),
        "Percentage Of Total": (pygsheets.FormatType.PERCENT, "0.0%"),
        "Total After Decimal (stream)": (pygsheets.FormatType.NUMBER, "00"),
        "Minutes (stream)": (pygsheets.FormatType.NUMBER, "0.0"),
        "Percentage Of Total (stream)": (pygsheets.FormatType.PERCENT, "0.0%"),
    },
    {
        "Total After Decimal": 125,
        "Minutes": 100,
        "Percentage Of Total": 150,
        "Total After Decimal (stream)": 225,
        "Minutes (stream)": 150,
        "Percentage Of Total (stream)": 225,
    },
)